In [436]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
from sklearn.feature_extraction.text import CountVectorizer

In [437]:
all_verbs = pd.read_csv('./all_verbs_LR.csv').reset_index(drop = True)
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [438]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [439]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e
...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ
33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ
33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ


In [440]:
all_verbs[~all_verbs['Generated Verb-SR'].str.contains('ˈ')].head(30)

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion


In [441]:
all_verbs['Stress Number'] = all_verbs['Generated Verb-SR'].apply(lambda word: word.rindex('ˈ') / len(word)) #before : goes input variable
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692
...,...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455
33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000
33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000


In [442]:
from scipy.sparse import hstack
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X = count_vect.fit_transform(all_verbs['Generated Verb-SR'])


In [443]:
hstack([X, all_verbs['Stress Number'].values.reshape(-1,1)]) 

<33872x1670 sparse matrix of type '<class 'numpy.float64'>'
	with 1221815 stored elements in COOrdinate format>

In [444]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Stress Number']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Stress Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Stress Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        #word_predict = model.predict(count_vect.transform(np.array(['ʌbɛmdinijɛst*ɛ*vijɛ'])))
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [445]:
log_with_stress_feature = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [446]:
log_with_stress_feature

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9911433172302737,
  'confusion matrix': array([[ 187,    0,   59],
         [   2, 7815,    2],
         [  32,    4, 3077]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.975398103417427,
  'confusion matrix': array([[3113,    5,    0,    0],
         [  11, 3258,  123,    0],
         [   0,  136, 2957,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994632313472893,
  'confusion matrix': array([[5641,    3],
         [   3, 5531]])},
 'Person': {'model': LogisticRegression(max_iter=100000

In [447]:
all_verbs['First Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: 0 if word[0]in ['a', 'o', 'u', 'i','ø', 'ɛ', 'ʌ', 'e'] else 1) #before : goes input variable

In [448]:
all_verbs[all_verbs['First Character'] == 1]

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character
200,200,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,me-*ne*-andʌd͡ʒ-i-em(ɛ),menaˈndʌd͡ʒijemɛ,a,0.250000,1
201,201,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,me-*ne*-andʌd͡ʒ-i-i,menaˈndʌd͡ʒii,a,0.307692,1
202,202,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,me-*ne*-andʌd͡ʒ-i-e,menaˈndʌd͡ʒije,a,0.285714,1
203,203,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,me-*ne*-andʌd͡ʒ-i-iɛ,menaˈndʌd͡ʒiijɛ,a,0.266667,1
204,204,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,me-*ne*-andʌd͡ʒ-i-om(ɛ),menaˈndʌd͡ʒijomɛ,a,0.250000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31561,31561,past,subjunctive,perfect,non-causative,affirmative,3sm,passive,intrans,be-van-i-ast-*abe*,bevɛnijastaˈbe,a,0.785714,1
31562,31562,past,subjunctive,perfect,non-causative,affirmative,3sf,passive,intrans,be-van-i-ast-*abiɛ*,bevɛnijɛstɛˈbijɛ,ɛ,0.687500,1
31563,31563,past,subjunctive,perfect,non-causative,affirmative,1p,passive,intrans,be-van-i-ast-*abimon*,bevɛnijɛstɛˈbimon,ɛ,0.647059,1
31564,31564,past,subjunctive,perfect,non-causative,affirmative,2p,passive,intrans,be-van-i-ast-*abion*,bevɛnijɛstɛˈbijon,ɛ,0.647059,1


In [449]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:


        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'First Character']] 
        y = all_verbs[feature]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [450]:
log_with_first_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [451]:
log_with_first_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9913222401145106,
  'confusion matrix': array([[ 189,    0,   57],
         [   2, 7816,    1],
         [  32,    5, 3076]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9756664877437824,
  'confusion matrix': array([[3114,    4,    0,    0],
         [  13, 3258,  121,    0],
         [   0,  134, 2959,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994632313472893,
  'confusion matrix': array([[5641,    3],
         [   3, 5531]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [452]:
all_verbs['Last Character'] = all_verbs['Generated Verb-SR'].apply(lambda word: word[-1]) #before : goes input variable

In [453]:
le = LabelEncoder()

all_verbs['Last Character Number'] = le.fit_transform(all_verbs['Last Character'])

In [454]:
all_verbs['Last Character Number']

0        15
1         5
2         3
3        15
4        15
         ..
33867    15
33868    15
33869     8
33870     8
33871     8
Name: Last Character Number, Length: 33872, dtype: int64

In [455]:
le = LabelEncoder()
all_verbs_encoded = all_verbs
all_verbs_encoded = all_verbs_encoded.apply(lambda col: le.fit_transform(col))

In [456]:
all_verbs_encoded

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number
0,0,1,1,0,1,1,1,1,0,9594,67,1,40,0,15,15
1,1,1,1,0,1,1,3,1,0,9596,62,1,46,0,5,5
2,2,1,1,0,1,1,6,1,0,9593,66,1,43,0,3,3
3,3,1,1,0,1,1,5,1,0,9597,63,1,41,0,15,15
4,4,1,1,0,1,1,0,1,0,9598,69,1,40,0,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,0,2,2,1,0,6,0,1,30435,30363,6,60,0,15,15
33868,33868,0,2,2,1,0,5,0,1,30435,30363,6,60,0,15,15
33869,33869,0,2,2,1,0,0,0,1,30433,30360,6,55,0,8,8
33870,33870,0,2,2,1,0,2,0,1,30434,30359,6,55,0,8,8


In [457]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000) 

        X = all_verbs[['Generated Verb-SR', 'Last Character Number']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [458]:
logistic_reg_with_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [459]:
logistic_reg_with_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9909643943460369,
  'confusion matrix': array([[ 187,    0,   59],
         [   2, 7816,    1],
         [  34,    5, 3074]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9760243335122563,
  'confusion matrix': array([[3114,    4,    0,    0],
         [  12, 3259,  121,    0],
         [   0,  131, 2962,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9992843084630524,
  'confusion matrix': array([[5640,    4],
         [   4, 5530]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [460]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:
        model = LogisticRegression(max_iter=100000)
        X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train)
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info


In [461]:
log_reg_with_first_and_last_character = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [462]:
log_reg_with_first_and_last_character

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9910538557881553,
  'confusion matrix': array([[ 190,    0,   56],
         [   2, 7816,    1],
         [  36,    5, 3072]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9760243335122563,
  'confusion matrix': array([[3114,    4,    0,    0],
         [  12, 3258,  122,    0],
         [   0,  130, 2963,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9992843084630524,
  'confusion matrix': array([[5640,    4],
         [   4, 5530]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [463]:
# as a last feature, I want to see if the number of vowels is relevant to the predictions

In [464]:
all_verbs['Vowels Quantity'] = all_verbs['Generated Verb-SR'].apply(lambda word: len(re.findall(r'[aouiøɛʌe]', word))) #before : goes input variable

In [465]:
all_verbs['Vowels Quantity']

0        6
1        5
2        5
3        6
4        6
        ..
33867    5
33868    5
33869    5
33870    5
33871    5
Name: Vowels Quantity, Length: 33872, dtype: int64

In [466]:
all_verbs

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33867,33867,past,subjunctive,perfect,non-causative,affirmative,3sm,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
33868,33868,past,subjunctive,perfect,non-causative,affirmative,3sf,active,trans,ʌ-χʌst-*abiʃ(ɛ)*,ʌχʌstɛˈbiʃɛ,ɛ,0.545455,0,ɛ,15,5
33869,33869,past,subjunctive,perfect,non-causative,affirmative,1p,active,trans,ʌ-χʌst-*abimon*,ʌχʌstɛˈbimon,ɛ,0.500000,0,n,8,5
33870,33870,past,subjunctive,perfect,non-causative,affirmative,2p,active,trans,ʌ-χʌst-*abion*,ʌχʌstɛˈbijon,ɛ,0.500000,0,n,8,5


In [467]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    for feature in features:

        model = LogisticRegression(max_iter=100000) 
        X = all_verbs[['Generated Verb-SR', 'Vowels Quantity']] 
        y = all_verbs[feature] #predicted/target varuiable
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 1)
        count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
        X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
        X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Vowels Quantity'].values.reshape(-1,1)]) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train) 
        y_test = le.transform(y_test)
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
  


In [468]:
log_reg_with_vowel_quantity = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [469]:
log_reg_with_vowel_quantity

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9912327786723922,
  'confusion matrix': array([[ 190,    0,   56],
         [   2, 7816,    1],
         [  34,    5, 3074]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9757559491859009,
  'confusion matrix': array([[3113,    5,    0,    0],
         [  13, 3260,  119,    0],
         [   0,  134, 2959,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9994632313472893,
  'confusion matrix': array([[5641,    3],
         [   3, 5531]])},
 'Person': {'model': LogisticRegression(max_iter=10000

In [470]:
## putting all features together in a LR model:

In [471]:

def logistic_regression_models():
    features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
    model_info = {}
    X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
    y = all_verbs[features] 
    X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X, y, test_size = 0.33, random_state = 1)
    count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
    X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
    X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
    for feature in features:
        model = LogisticRegression(max_iter=100000) 
        le = LabelEncoder()
        y_train = le.fit_transform(y_train_pre_encoded[feature]) 
        y_test = le.transform(y_test_pre_encoded[feature])
        print(le.classes_)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        matrix = confusion_matrix(y_test, y_pred)
        accuracy = accuracy_score(y_test, y_pred)
        model_info[feature] = { 'model': model,'accuracy': accuracy, 'confusion matrix': matrix }
        
    return model_info
    # this code is imroved: most reliable for my classification models (it is faster as some parts have gone out of the scope of the function)


In [472]:
log_reg_with_three_added_features = logistic_regression_models()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [473]:
log_reg_with_three_added_features

{'Tense': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9993737699051709,
  'confusion matrix': array([[7813,    2],
         [   5, 3358]])},
 'Modality': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.991411701556629,
  'confusion matrix': array([[ 193,    0,   53],
         [   2, 7816,    1],
         [  35,    5, 3073]])},
 'Aspect': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9759348720701377,
  'confusion matrix': array([[3113,    5,    0,    0],
         [  12, 3258,  122,    0],
         [   0,  130, 2963,    0],
         [   0,    0,    0, 1575]])},
 'Causation': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9999105385578816,
  'confusion matrix': array([[3527,    0],
         [   1, 7650]])},
 'Polarity': {'model': LogisticRegression(max_iter=100000),
  'accuracy': 0.9992843084630524,
  'confusion matrix': array([[5640,    4],
         [   4, 5530]])},
 'Person': {'model': LogisticRegression(max_iter=100000

In [474]:
all_verbs.head()

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
0,0,present,indicative,imperfective,non-causative,negative,1s,passive,intrans,a-me-*ne*-ger-i-em(ɛ),amneˈgerijemɛ,e,0.307692,0,ɛ,15,6
1,1,present,indicative,imperfective,non-causative,negative,2s,passive,intrans,a-me-*ne*-ger-i-i,amneˈgerii,e,0.400000,0,i,5,5
2,2,present,indicative,imperfective,non-causative,negative,3sm,passive,intrans,a-me-*ne*-ger-i-e,amneˈgerije,e,0.363636,0,e,3,5
3,3,present,indicative,imperfective,non-causative,negative,3sf,passive,intrans,a-me-*ne*-ger-i-iɛ,amneˈgeriijɛ,e,0.333333,0,ɛ,15,6
4,4,present,indicative,imperfective,non-causative,negative,1p,passive,intrans,a-me-*ne*-ger-i-om(ɛ),amneˈgerijomɛ,e,0.307692,0,ɛ,15,6


In [475]:
tense_model = log_reg_with_three_added_features['Tense']['model'] #separating the models from each other

In [476]:
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] #select the list by two square brackets
y = all_verbs['Tense'] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) # splitting X twice , in one of them keep the original in one vectorized. 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3))  
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded) 
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)

['past' 'present']


In [477]:
# I only pulled out tense, and now we are predict on the test set top see where the misclassifications occured
tense_model.predict(X_test) # compare these with y_test

array([1, 0, 0, ..., 1, 0, 1])

In [478]:
y_test

array([1, 0, 0, ..., 1, 0, 1])

In [479]:
y_test !=  tense_model.predict(X_test) # pairwise comparison

array([False, False, False, ..., False, False, False])

In [480]:
words_test[y_test !=  tense_model.predict(X_test)] # these will keep only the rows that do not agree

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity
8764,mømaˈndɛ,15,1,3
11548,neˈrbetemɛ,15,1,4
1130,meneˈbardemɛ,15,1,5
11549,neˈrbeti,5,1,3
11555,beˈrbetemɛ,15,1,4
11538,merbetiˈ,19,1,3
11556,beˈrbeti,5,1,3



aspect_model = log_reg_with_three_added_features['Aspect']['model']
X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
le = LabelEncoder()
y_train = le.fit_transform(y_train_pre_encoded)
y_test = le.transform(y_test_pre_encoded)
print(le.classes_)
aspect_model.predict(X_test)
aspect_mismatches = words_test[y_test !=  aspect_model.predict(X_test)]
aspect_mismatches['y_train_pre_encoded'] = y_train_pre_encoded
aspect_mismatches.head()

In [481]:
type(words_test)

pandas.core.frame.DataFrame

In [482]:
type(X_test)

scipy.sparse.coo.coo_matrix

In [483]:
tense_model.predict(X_test)

array([1, 0, 0, ..., 1, 0, 1])

In [484]:
le.classes_

array(['past', 'present'], dtype=object)

words_test['predicted'] = modaliy_model.predict(X_test) #made a new column

words_test['predicted_aspect'] = words_test['predicted'].apply(lambda x: le.classes_[x]) #index lookup and doing the opposite of vectorization. Give me the class at index 0 or 1 ...

words_test['actual_aspect'] = all_verbs.iloc[words_test.index]['Aspect'] #word test preserves the indexes from the original data frame (see on the left): the indexe are not in a paarticular order. so you can use this index to go back to 

original df and pass that indexes to df and look up what the values are for the rows with that index, we only care about values of aspect columns, we wanna get the aspect value


In [485]:
def LG_model():
        dataframes = {}
        features = ['Tense', 'Modality', 'Aspect', 'Causation', 'Polarity', 'Person', 'Voice', 'Transitivity']
        for feature in features:
                feature_model = log_reg_with_three_added_features[feature]['model']
                X = all_verbs[['Generated Verb-SR', 'Last Character Number', 'First Character', 'Vowels Quantity']] 
                y = all_verbs[feature]
                words_train, words_test, X_train, X_test, y_train_pre_encoded, y_test_pre_encoded = train_test_split(X,X, y, test_size = 0.33, random_state = 1) 
                count_vect = CountVectorizer(token_pattern= '.', ngram_range= (1,3)) 
                X_train = hstack([count_vect.fit_transform(X_train['Generated Verb-SR']), X_train['Last Character Number'].values.reshape(-1,1), X_train['First Character'].values.reshape(-1,1), X_train['Vowels Quantity'].values.reshape(-1,1)]) 
                    
                X_test = hstack([count_vect.transform(X_test['Generated Verb-SR']), X_test['Last Character Number'].values.reshape(-1,1), X_test['First Character'].values.reshape(-1,1), X_test['Vowels Quantity'].values.reshape(-1,1)]) #no need to fit again, otherwise what it sees on training and test data may differ and end up with an erorr
                le = LabelEncoder()
                y_train = le.fit_transform(y_train_pre_encoded)
                y_test = le.transform(y_test_pre_encoded)
                print(le.classes_)
                words_test['predicted'] = feature_model.predict(X_test) 
                words_test['predicted ' + feature] = words_test['predicted'].apply(lambda x: le.classes_[x])
                words_test['actual ' + feature] = pd.Series(y_test).apply(lambda x: le.classes_[x]).values
                dataframes[feature] = words_test
        return dataframes


In [486]:
dfs = LG_model()

['past' 'present']
['imperative' 'indicative' 'subjunctive']
['imperfective' 'neutral' 'perfect' 'pluperfect']
['causative' 'non-causative']
['affirmative' 'negative']
['1p' '1s' '2p' '2s' '3p' '3sf' '3sm']
['active' 'passive']
['intrans' 'trans']


In [487]:
dfs['Tense']

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense
22176,nʌˈɢʌnomɛ,15,1,4,1,present,present
16416,bet͡ʃ:ɛrdinijɛstijʌˈ,19,1,7,0,past,past
3161,begɛrdinijaˈst,11,1,5,0,past,past
5860,neˈkalʌʃijast,11,1,5,0,past,past
31920,ʌvid͡ʒijaˈstɛ,15,0,5,0,past,past
...,...,...,...,...,...,...,...
28091,ʌˈniʃdinije,3,0,5,1,present,present
27952,ʌneˈndʌstɛbimɛ,15,0,6,0,past,past
8136,memʌsdiniˈjɛ,15,1,5,1,present,present
22024,devʌzijɛstijʌˈ,19,1,6,0,past,past


In [488]:
base_df = dfs['Tense'].copy()
base_df['predicted Modality'] = dfs['Modality']['predicted Modality']
base_df['actual Modality'] = dfs['Modality']['actual Modality']
base_df['predicted Aspect'] = dfs['Aspect']['predicted Aspect']
base_df['actual Aspect'] = dfs['Aspect']['actual Aspect']
base_df['predicted Causation'] = dfs['Causation']['predicted Causation']
base_df['actual Causation'] = dfs['Causation']['actual Causation']
base_df['predicted Polarity'] = dfs['Polarity']['predicted Polarity']
base_df['actual Polarity'] = dfs['Polarity']['actual Polarity']
base_df['predicted Person'] = dfs['Person']['predicted Person']
base_df['actual Person'] = dfs['Person']['actual Person']
base_df['predicted Transitivity'] = dfs['Transitivity']['predicted Transitivity']
base_df['actual Transitivity'] = dfs['Transitivity']['actual Transitivity']

In [489]:
base_df.shape

(11178, 19)

In [490]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
22176,nʌˈɢʌnomɛ,15,1,4,1,present,present,subjunctive,subjunctive,neutral,neutral,non-causative,non-causative,negative,negative,1p,1p,trans,trans
16416,bet͡ʃ:ɛrdinijɛstijʌˈ,19,1,7,0,past,past,indicative,indicative,perfect,perfect,causative,causative,affirmative,affirmative,3sf,3sf,intrans,intrans
3161,begɛrdinijaˈst,11,1,5,0,past,past,indicative,indicative,neutral,neutral,causative,causative,affirmative,affirmative,3sm,3sm,intrans,intrans
5860,neˈkalʌʃijast,11,1,5,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,3sm,3sm,intrans,intrans
31920,ʌvid͡ʒijaˈstɛ,15,0,5,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,affirmative,affirmative,3sf,3sf,intrans,intrans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28091,ʌˈniʃdinije,3,0,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,affirmative,affirmative,3sm,3sm,intrans,intrans
27952,ʌneˈndʌstɛbimɛ,15,0,6,0,past,past,subjunctive,subjunctive,perfect,perfect,non-causative,non-causative,negative,negative,1s,1s,trans,trans
8136,memʌsdiniˈjɛ,15,1,5,1,present,present,indicative,indicative,imperfective,imperfective,causative,causative,affirmative,affirmative,3sf,3sf,trans,trans
22024,devʌzijɛstijʌˈ,19,1,6,0,past,past,indicative,indicative,perfect,perfect,non-causative,non-causative,affirmative,affirmative,3sf,3sf,intrans,intrans


In [491]:
base_df.to_csv('./base_df.csv', encoding = 'utf-8-sig', index = False)

In [492]:
base_df

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Tense,actual Tense,predicted Modality,actual Modality,predicted Aspect,actual Aspect,predicted Causation,actual Causation,predicted Polarity,actual Polarity,predicted Person,actual Person,predicted Transitivity,actual Transitivity
22176,nʌˈɢʌnomɛ,15,1,4,1,present,present,subjunctive,subjunctive,neutral,neutral,non-causative,non-causative,negative,negative,1p,1p,trans,trans
16416,bet͡ʃ:ɛrdinijɛstijʌˈ,19,1,7,0,past,past,indicative,indicative,perfect,perfect,causative,causative,affirmative,affirmative,3sf,3sf,intrans,intrans
3161,begɛrdinijaˈst,11,1,5,0,past,past,indicative,indicative,neutral,neutral,causative,causative,affirmative,affirmative,3sm,3sm,intrans,intrans
5860,neˈkalʌʃijast,11,1,5,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,negative,negative,3sm,3sm,intrans,intrans
31920,ʌvid͡ʒijaˈstɛ,15,0,5,0,past,past,indicative,indicative,neutral,neutral,non-causative,non-causative,affirmative,affirmative,3sf,3sf,intrans,intrans
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28091,ʌˈniʃdinije,3,0,5,1,present,present,subjunctive,subjunctive,neutral,neutral,causative,causative,affirmative,affirmative,3sm,3sm,intrans,intrans
27952,ʌneˈndʌstɛbimɛ,15,0,6,0,past,past,subjunctive,subjunctive,perfect,perfect,non-causative,non-causative,negative,negative,1s,1s,trans,trans
8136,memʌsdiniˈjɛ,15,1,5,1,present,present,indicative,indicative,imperfective,imperfective,causative,causative,affirmative,affirmative,3sf,3sf,trans,trans
22024,devʌzijɛstijʌˈ,19,1,6,0,past,past,indicative,indicative,perfect,perfect,non-causative,non-causative,affirmative,affirmative,3sf,3sf,intrans,intrans


In [493]:
aspect_df = dfs['Aspect']

In [494]:
aspect_df[aspect_df['Generated Verb-SR'] == 'ʌriˈtemɛ']

,Generated Verb-SR,Last Character Number,First Character,Vowels Quantity,predicted,predicted Aspect,actual Aspect
30023,ʌriˈtemɛ,15,0,4,1,neutral,neutral


In [495]:
all_verbs[all_verbs['Generated Verb-SR'] == 'ʌriˈtemɛ']

,Unnamed: 0,Tense,Modality,Aspect,Causation,Polarity,Person,Voice,Transitivity,Generated Verb-UR,Generated Verb-SR,Stressed Portion,Stress Number,First Character,Last Character,Last Character Number,Vowels Quantity
30023,30023,past,indicative,neutral,non-causative,affirmative,1s,active,trans,ʌ-*rit*-em(ɛ),ʌriˈtemɛ,i,0.375,0,ɛ,15,4
